# Input

Time series $Z$ mixed with noise
$$Y = Z + E,$$
where $E$ is a whitenoise
$$
cov(E_i, E_j) =
\begin{cases}
\sigma_e^2,& i = j \\
0,&          i \neq j
\end{cases} 
$$

$Z$ depends on $X_i, i = 1..6$ time series.

# Task
- Find and remove anomalies in $Y$
- Create regression model $\mu$: $Z = \mu\left( X \right)$
- Find prediction for three more values of $Z$ by
  - extrapolation of $Z$
  - model $\mu\left( X \right)$ and prediction of $X$
- Estimate prediction errors
- Choose the best prediction

# Tools
- [Exponential smoothing](https://en.wikipedia.org/wiki/Exponential_smoothing)
- [Autoregressive model](https://en.wikipedia.org/wiki/Autoregressive_model)